<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train8_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:

train8 = pd.read_csv(r'/content/train_data8.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [3]:

train8['Target'] = train8['Target'].replace(['Others'],'Invalid')


In [4]:
train8['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [5]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train8_Xtfidf = tfidf_vect.fit_transform(train8['Sentence'])


In [7]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [8]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [9]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [10]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [11]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [12]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train8_Xtfidf,train8['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6858407079646017
                  precision    recall  f1-score   support

        Analysis       0.52      0.40      0.45       127
      Conclusion       0.72      0.51      0.60        45
           Facts       0.76      0.92      0.83       486
         Invalid       0.52      0.48      0.50       130
           Issue       0.55      0.47      0.51        49
Rule/Law/Holding       0.67      0.21      0.32        67

        accuracy                           0.69       904
       macro avg       0.62      0.50      0.53       904
    weighted avg       0.67      0.69      0.66       904



In [14]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [15]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
[0.992177684577018, 0.9902468028280652, 0.9926236407159031, 0.9955428078937424, 0.9998972292792527, 0.9981722446728436, 0.9968409280802937, 0.9976122938240191, 0.9954168813588864, 0.9981814401683025, 0.9889960268637619, 0.9944759493543492, 0.9982185533760509, 0.9982093659404663, 0.9992914945414413, 0.9919899135068841, 0.9971309244601273, 0.9952576351746895, 0.9857115688191357, 0.9970244317356123, 0.999294237674347, 0.9997388870788562, 0.9845840736107782, 0.9845565956524599, 0.992401514231481, 0.992660158119536, 0.9939660280076908, 0.9994732071625134, 0.9952575813302115, 0.9802433676364993, 0.9997735829199755, 0.9906549747905413, 0.9821700350319088, 0.9881503861207221, 0.9886704943214706, 0.9938145711886003, 0.992917206881906, 0.9994930227395947, 0.9995303781789169, 0.9976973595864923, 0.997406858872675]
[0, 1, 2, 4, 5, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 3

In [16]:
unlabel_1=unlabel_1.loc[[0, 1, 2, 4, 5, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 31, 35, 37, 39, 45, 48, 49, 51, 55, 56, 60, 62, 70, 73, 76, 77, 79, 84, 87, 89, 93, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts


In [17]:
frame_1 = [train8,unlabel_1]
train8_1 = pd.concat(frame_1)
len(train8_1)

3517

In [18]:
x_train_1 = tfidf_vect.transform(train8_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train8_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6590909090909091
                  precision    recall  f1-score   support

        Analysis       0.44      0.42      0.43        98
      Conclusion       0.70      0.50      0.58        38
           Facts       0.75      0.89      0.81       360
         Invalid       0.50      0.46      0.48       111
           Issue       0.62      0.49      0.55        37
Rule/Law/Holding       0.58      0.23      0.33        60

        accuracy                           0.66       704
       macro avg       0.60      0.50      0.53       704
    weighted avg       0.64      0.66      0.64       704



In [19]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9931547334430195, 0.9990147561567955, 0.9899367257049411, 0.9931563266276381, 0.9987166593174859, 0.9992546403437855, 0.9999497836338729, 0.9865776146918946, 0.9819829780773054, 0.9909592232713258, 0.98764049483993, 0.9932463462332421, 0.9877957431305473, 0.9989377693113993, 0.9929770516684946, 0.9962418736928443, 0.9943290091170072, 0.9980360843499373, 0.9999394560276847, 0.9989439326423138, 0.9998246352272967, 0.9934701808593854, 0.9994526668662636, 0.9819334322869062, 0.9938130785665866, 0.999263137107741, 0.9922187488399514, 0.999196641735043, 0.9997026878888047, 0.9984516000494879, 0.9981348668968731, 0.9876476080035861, 0.990168640941328, 0.995213461799368, 0.9988199650976842, 0.998197031463825, 0.9963907869434502, 0.9994489847012931, 0.9977455734391542, 0.9918301851119417, 0.9931966167793305, 0.9940173109910412]
[0, 2, 3, 4, 7, 8, 12, 13, 16, 17, 20, 

In [20]:
unlabel_2 = unlabel_2.loc[[0, 2, 3, 4, 7, 8, 12, 13, 16, 17, 20, 21, 23, 29, 31, 32, 34, 36, 37, 39, 42, 44, 49, 52, 55, 59, 60, 63, 64, 69, 70, 71, 75, 77, 79, 83, 85, 88, 90, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [21]:
frame_2 = [train8_1,unlabel_2]
train8_2 = pd.concat(frame_2)
len(train8_2)

3559

In [22]:
x_train_2 = tfidf_vect.transform(train8_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train8_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6566077003121749
                  precision    recall  f1-score   support

        Analysis       0.49      0.41      0.44       145
      Conclusion       0.70      0.37      0.49        51
           Facts       0.74      0.89      0.81       502
         Invalid       0.45      0.45      0.45       144
           Issue       0.71      0.36      0.48        61
Rule/Law/Holding       0.54      0.33      0.41        58

        accuracy                           0.66       961
       macro avg       0.61      0.47      0.51       961
    weighted avg       0.64      0.66      0.64       961



In [23]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9960181212393071, 0.9883766237644633, 0.997262520502243, 0.9936216702147401, 0.9993876013307127, 0.9866530304641552, 0.9995099642444591, 0.9806156766475312, 0.9810083712920925, 0.9983709496582306, 0.9885546123138492, 0.9966563135090276, 0.9994779240190722, 0.9820941243965577, 0.9956117251167731, 0.9942364800973092, 0.9986223064634603, 0.98924961920921, 0.9980740945330739, 0.9984693365759876, 0.9982115001057832, 0.9985876636079927, 0.9970910442292125, 0.9821822923364496, 0.996999084294604, 0.9941847732860565, 0.9881507247813901, 0.9932005563381551, 0.9996382842895926, 0.996686129417714, 0.9884245679344017, 0.9994020832907494, 0.9987816484546458, 0.9996595263961471, 0.9945932640845838, 0.9814740021136087, 0.9885953091729458, 0.9988118335962379, 0.9989599655917301, 0.9936410454658076]
[0, 2, 3, 7, 8, 11, 12, 13, 19, 20, 22, 26, 27, 29, 30, 35, 36, 41, 42, 43, 46, 49,

In [24]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 7, 8, 11, 12, 13, 19, 20, 22, 26, 27, 29, 30, 35, 36, 41, 42, 43, 46, 49, 51, 52, 54, 56, 61, 63, 65, 66, 67, 68, 70, 72, 75, 78, 79, 89, 96, 97],:]
frame_3 = [train8_2,unlabel_3]
train8_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train8_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train8_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6708333333333333
                  precision    recall  f1-score   support

        Analysis       0.49      0.43      0.46       101
      Conclusion       0.75      0.57      0.65        37
           Facts       0.75      0.89      0.81       375
         Invalid       0.50      0.45      0.47       121
           Issue       0.66      0.45      0.54        42
Rule/Law/Holding       0.56      0.32      0.41        44

        accuracy                           0.67       720
       macro avg       0.62      0.52      0.55       720
    weighted avg       0.65      0.67      0.66       720



In [25]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 4, 2, 2, 2, 0, 2, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 3, 2, 3, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2]
[0.9960721236399966, 0.9991036502513159, 0.9992910017966136, 0.9910369179514439, 0.9996973259541526, 0.9826243303341751, 0.997996812431864, 0.9994690959585607, 0.9971409789465522, 0.9903246679991963, 0.9990142583726103, 0.9969471792447976, 0.9987836821193646, 0.9984477397044879, 0.9997161500155866, 0.9989928587111073, 0.9996546282462013, 0.9937296237874143, 0.9976336759451889, 0.9927028455665745, 0.9981828735494822, 0.9886998029717889, 0.9940856143486964, 0.9933714019888145, 0.9985056626726372, 0.9956516080924953, 0.9991418409696255, 0.9993190502507862, 0.9862648605947638, 0.9999723389764191, 0.9851566182156734, 0.9901063411116905, 0.9924901613183096, 0.9975754329344609, 0.9960169343487113, 0.9926492812420266]
[2, 4, 7, 8, 10, 12, 15, 18, 22, 25, 27, 28, 29, 32, 34, 37, 40, 42, 46, 50, 56, 58, 64, 66, 67, 69, 70, 71, 72, 75, 79, 82, 85, 87, 97, 99]
36
36
36


In [26]:
unlabel_4=unlabel_4.loc[[2, 4, 7, 8, 10, 12, 15, 18, 22, 25, 27, 28, 29, 32, 34, 37, 40, 42, 46, 50, 56, 58, 64, 66, 67, 69, 70, 71, 72, 75, 79, 82, 85, 87, 97, 99],:]

In [27]:
frame_4 = [train8_3,unlabel_4]
train8_4 = pd.concat(frame_4)
len(train8_4)
x_train_4 = tfidf_vect.transform(train8_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train8_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6883910386965377
                  precision    recall  f1-score   support

        Analysis       0.52      0.44      0.48       145
      Conclusion       0.67      0.47      0.55        59
           Facts       0.79      0.90      0.84       509
         Invalid       0.50      0.51      0.51       148
           Issue       0.55      0.50      0.53        52
Rule/Law/Holding       0.63      0.35      0.45        69

        accuracy                           0.69       982
       macro avg       0.61      0.53      0.56       982
    weighted avg       0.67      0.69      0.67       982



In [28]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0]
[0.9991305859239628, 0.9882495848741578, 0.994794101520088, 0.984842210965426, 0.9931050847149414, 0.9983982584895529, 0.9974718332209563, 0.9975214280024969, 0.9995148603763799, 0.9836476116520891, 0.9946448193572177, 0.9997987118089527, 0.9942663540725345, 0.9984495371723117, 0.9997066886147153, 0.9810436695827595, 0.997605433941649, 0.9888386714547713, 0.9896969359749976, 0.9995214696525148, 0.9949456493449697, 0.9980893156204519, 0.9944546601189245, 0.9952159591288284, 0.9811867446728005, 0.9956801519971068, 0.9909092657503932, 0.9998966090679744, 0.9992844901745116, 0.9895206779166253, 0.9931422022283715, 0.9992388557909933, 0.9981804864151572, 0.9932281876542947, 0.9910254758313813, 0.9933548091233868, 0.9960091327078933, 0.9901614262127945]
[3, 7, 8, 10, 12, 19, 22, 23, 26, 27, 32, 36, 46, 52, 53, 59, 60, 62, 63, 65, 66, 73, 75, 76, 77, 78, 81, 82, 86, 92, 93, 94, 9

In [29]:
unlabel_5 = unlabel_5.loc[[3, 7, 8, 10, 12, 19, 22, 23, 26, 27, 32, 36, 46, 52, 53, 59, 60, 62, 63, 65, 66, 73, 75, 76, 77, 78, 81, 82, 86, 92, 93, 94, 95, 97, 100, 103, 104, 110],:]

In [30]:
frame_5 = [train8_4,unlabel_5]
train8_5 = pd.concat(frame_5)
len(train8_5)
x_train_5 = tfidf_vect.transform(train8_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train8_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6643145161290323
                  precision    recall  f1-score   support

        Analysis       0.50      0.41      0.45       138
      Conclusion       0.59      0.50      0.54        52
           Facts       0.78      0.87      0.82       528
         Invalid       0.45      0.48      0.47       155
           Issue       0.57      0.51      0.54        53
Rule/Law/Holding       0.42      0.21      0.28        66

        accuracy                           0.66       992
       macro avg       0.55      0.50      0.52       992
    weighted avg       0.65      0.66      0.65       992



In [31]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.690978886756238


In [32]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6316170485288132, 0.5942271915523814, 0.6031395474151282, None)
